## Training BERT large

This is just one run that we repeat 10 times for each dataset (for that change the first code cell and the test dataset load cell and report the mean value of each.

For the purpose of debugging, this notebook has been run with both bert large (results reported in our paper) and with distilbert (for smaller gpus).

In [4]:
import pandas as pd
# we create temporary file for the dataset, based on what kind of train set we want
prompt_0 = pd.read_csv('gpt_datasets/for_train_test/prompt_h_0.csv').reset_index(drop=True)
prompt_gpt_0 = pd.read_csv('gpt_datasets/for_train_test/prompt_gpt_0.csv').reset_index(drop=True)

prompt_12 = pd.read_csv('gpt_datasets/for_train_test/prompt_h_12.csv').reset_index(drop=True)
taboo_12 = pd.read_csv('gpt_datasets/for_train_test/taboo_h_12.csv').reset_index(drop=True)
prompt_gpt_12 = pd.read_csv('gpt_datasets/for_train_test/prompt_gpt_12.csv').reset_index(drop=True)
taboo_gpt_12 = pd.read_csv('gpt_datasets/for_train_test/taboo_gpt_12.csv').reset_index(drop=True)
taboo_gpt_12_with_filt = pd.read_csv('gpt_datasets/for_train_test/taboo_gpt_12_filt.csv').reset_index(drop=True)

In [5]:
from sklearn.model_selection import train_test_split
# replace this for whatever file you want to use in your experiments
train_taboo_gpt, test_taboo_gpt = train_test_split(taboo_gpt_12_with_filt, test_size=0.2) # can be exchanged for the split with tabooed samples
train_taboo_gpt = train_taboo_gpt.append(prompt_gpt_0)

train_taboo_gpt.to_csv('gpt_datasets/for_train_test/tmp/train.csv')
test_taboo_gpt.to_csv('gpt_datasets/for_train_test/tmp/test.csv')

In [8]:
from datasets import load_dataset

data_files = {"train": "gpt_datasets/for_train_test/tmp/train.csv", "test": "gpt_datasets/for_train_test/tmp/test.csv"}

dataset_train, dataset_test = load_dataset("csv", data_files=data_files, split=['train', 'test'], keep_default_na=False)

Using custom data configuration default-a4f44c53a2fe693f


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\jance\.cache\huggingface\datasets\csv\default-a4f44c53a2fe693f\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #bert-large-uncased


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset_train.map(tokenize_function, batched=True)
tokenized_test_datasets = dataset_test.map(tokenize_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=10)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

In [4]:
# only if you dont have the evaluate lib from huggingface, e.g. in the kaggle env
!pip install evaluate

In [11]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from joblib import dump, load
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
import numpy as np
import itertools

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="bert_taboo")

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="bert_taboo", evaluation_strategy="epoch", per_device_eval_batch_size=16, per_device_train_batch_size=8, learning_rate=1e-5, num_train_epochs=5, save_steps=10000)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_test_datasets,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Unnamed: 0, text.
***** Running training *****
  Num examples = 4488
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2805


Epoch,Training Loss,Validation Loss,Accuracy
1,1.047000,0.139199,0.975940
2,0.076600,0.052706,0.983459
3,0.025300,0.042809,0.986466
4,0.011800,0.038185,0.987970
5,0.009000,0.038144,0.987970


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Unnamed: 0, text.
***** Running Evaluation *****
  Num examples = 665
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Unnamed: 0, text.
***** Running Evaluation *****
  Num examples = 665
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Unnamed: 0, text.
***** Running Evaluation *****
  Num examples = 665
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Unnamed: 0, text.
***** Running Evaluation *****
  Num examples = 665
  Batch size = 16
The following columns in the evaluat

TrainOutput(global_step=2805, training_loss=0.20916806818542208, metrics={'train_runtime': 786.9204, 'train_samples_per_second': 28.516, 'train_steps_per_second': 3.565, 'total_flos': 2972992518144000.0, 'train_loss': 0.20916806818542208, 'epoch': 5.0})

In [14]:

dataset = load_dataset("csv", data_files="gpt_datasets/for_train_test/prompt_gpt_12.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.2)

dataset_prompt_gpt = dct_dataset['test']


dataset = load_dataset("csv", data_files="gpt_datasets/for_train_test/taboo_gpt_12.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.2)

dataset_taboo_human = dct_dataset['test']


dataset = load_dataset("csv", data_files="gpt_datasets/for_train_test/prompt_h_12.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.2)

dataset_prompt_human = dct_dataset['test']

Using custom data configuration default-a5d5869447a158ce
Reusing dataset csv (C:\Users\jance\.cache\huggingface\datasets\csv\default-a5d5869447a158ce\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
Using custom data configuration default-98f3985ad150b854
Reusing dataset csv (C:\Users\jance\.cache\huggingface\datasets\csv\default-98f3985ad150b854\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
Using custom data configuration default-7bf051d34ce84287
Reusing dataset csv (C:\Users\jance\.cache\huggingface\datasets\csv\default-7bf051d34ce84287\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


In [15]:
tokenized_dataset_all_gpt = dataset_prompt_gpt.map(tokenize_function, batched=True)
tokenized_dataset_taboo_h = dataset_taboo_human.map(tokenize_function, batched=True)
tokenized_dataset_all = dataset_prompt_human.map(tokenize_function, batched=True)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [16]:
trainer.evaluate(tokenized_dataset_all_gpt)

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 697
  Batch size = 8


{'eval_loss': 0.0009535038843750954,
 'eval_accuracy': 1.0,
 'eval_runtime': 7.5322,
 'eval_samples_per_second': 92.536,
 'eval_steps_per_second': 11.683,
 'epoch': 5.0}

In [17]:
trainer.evaluate(tokenized_dataset_taboo_h)

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1188
  Batch size = 8


{'eval_loss': 0.2895137369632721,
 'eval_accuracy': 0.9326599326599326,
 'eval_runtime': 12.3836,
 'eval_samples_per_second': 95.934,
 'eval_steps_per_second': 12.032,
 'epoch': 5.0}

In [18]:
trainer.evaluate(tokenized_dataset_all)

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1130
  Batch size = 8


{'eval_loss': 0.1644696593284607,
 'eval_accuracy': 0.9690265486725663,
 'eval_runtime': 11.9438,
 'eval_samples_per_second': 94.61,
 'eval_steps_per_second': 11.889,
 'epoch': 5.0}